In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_data = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
test_data = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")

In [3]:
num_col = list(train_data.select_dtypes(include=['int','float']).columns)
num_col = num_col[:len(num_col)-1]

In [4]:
# Remove outlier for all the numeric columns
# Use Inter-Quartile Range (IQR) proximity rule.
# The data points which fall below Q1 – 1.5 IQR or above Q3 + 1.5 IQR are outliers.
# where Q1 and Q3 are the 25th and 75th percentile of the dataset respectively,
# and IQR represents the inter-quartile range and given by Q3 – Q1.
for col in num_col:
    q1 = train_data[col].quantile(0.25)
    q3 = train_data[col].quantile(0.75)
    iqr = q3 - q1
    upper_bound = q3 + 1.5 * (iqr)
    lower_bound = q1 - 1.5 * (iqr)
    train_data.loc[train_data[col] < lower_bound, col] = np.nan
    train_data.loc[train_data[col] > upper_bound, col] = np.nan
    mean = train_data[col].mean(skipna=True) 
    train_data[col] = train_data[col].fillna(mean)

In [5]:
train_y = train_data[['SalePrice']]
train_X = train_data.drop(columns=['Id', 'SalePrice'])
test_data = test_data.drop(columns=['Id'])

In [6]:
# Get object data columns and numeric data columns
obj_col = list(train_X.select_dtypes(include=['object']).columns)
num_col = list(train_X.select_dtypes(include=['int','float']).columns)

In [7]:
split = train_X.shape[0]

In [8]:
# Concatenate train_X and test_data to perform One-hot-encoding on categorical columns
data = pd.concat([train_X, test_data],ignore_index=True)

In [9]:
data[obj_col] = data[obj_col].fillna("0")

In [10]:
ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)
new_df = pd.DataFrame(ohe.fit_transform(data[obj_col]))
data = data.join(new_df)

In [11]:
data = data.drop(columns=obj_col)

In [12]:
# split train and test data
train_X = data.iloc[:split,:]
test_data = data.iloc[split:,:]

In [13]:
# Fill nan in numeric columns with the mean value of that column
for col in num_col:
    mean = train_X[col].mean(skipna=True)
    train_X[col] = train_X[col].fillna(mean)
    mean = test_data[col].mean(skipna=True)
    test_data[col] = test_data[col].fillna(mean)

In [14]:
# Converting dataframe to numpy array
X_train = train_X.to_numpy()
y_train = train_y.to_numpy()
X_test = test_data.to_numpy()

In [15]:
sfm = SelectFromModel(xgb.XGBRegressor())
sfm.fit(X_train, y_train)
sfm.get_support()
selected_col= train_X.columns[(sfm.get_support())]
selected_col

In [16]:
train_X = train_X[selected_col]
test_data = test_data[selected_col]

In [17]:
X_train = train_X.to_numpy()
X_test = test_data.to_numpy()

In [18]:
pca = PCA(2)

In [19]:
"""
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8)
"""

In [20]:
# Use grid search to get the optimal hyperparameters
"""
n_estimators = [100,200,300,400,500]
max_depth = [3,4,5,10]
eta = [0.01,0.1,0.3]

opt_dict = {'n_estimators': [100,200,300,400,500],
            'num_boost_round': [10, 25, 50],
            'eta': [0.01, 0.05, 0.1, 0.3],
            'max_depth': [3, 4, 5,10],
            'subsample': [0.9, 1.0],
            'colsample_bytree': [0.9, 1.0]}

model = xgb.XGBRegressor(eval_metric = 'rmse')

model = RandomizedSearchCV(model, opt_dict, n_jobs=1, cv=2)
model.fit(X_train,y_train)
print(model.best_score_)
print(model.best_params_)
"""

In [21]:
"""
n_estimators = [100,200,300,400,500]
max_depth = [3,4,5,10]
max_features = ["auto", "sqrt", "log2"]
criterion = ["squared_error", "absolute_error", "friedman_mse"]
learning_rate = [0.1, 1, 10]
loss = ['squared_error', 'absolute_error', 'quantile']


opt_dict = {'n_estimators':n_estimators,
            'criterion': criterion,
            'max_depth': max_depth,
            'max_features': max_features,
            'learning_rate': learning_rate,
            'loss': loss}

model = GradientBoostingRegressor()

model = RandomizedSearchCV(model, opt_dict, n_jobs=1, cv=2)
model.fit(X_train,y_train)
print(model.best_score_)
print(model.best_params_)
"""

In [22]:
# Apply optimal hyperparameters to the model
model = xgb.XGBRegressor(max_depth=3, n_estimators=500, eta=0.05, subsample=1.0, num_boost_round=25, colsample_bytree= 0.9)
#model = GradientBoostingRegressor(n_estimators=400, max_depth=10, max_features='log2', loss='squared_error', learning_rate=0.1, criterion='squared_error')
#estimators = [
#    ('gbr', GradientBoostingRegressor(n_estimators=400, max_depth=10, max_features='log2', loss='squared_error', learning_rate=0.1, criterion='squared_error')),
#    ('rfr', RandomForestRegressor(n_estimators=400, max_depth=10, max_features='auto', criterion='absolute_error')),
#    ('dtr', DecisionTreeRegressor(max_features='auto', max_depth=3, criterion='friedman_mse'))
#]
#reg = StackingRegressor(
#    estimators=estimators,
#   final_estimator=model
#)
#GradientBoostingRegressor(n_estimators=400, max_depth=10, max_features='log2', loss='squared_error', learning_rate=0.1, criterion='squared_error')
#RandomForestRegressor(n_estimators=400, max_depth=10, max_features='auto', criterion='absolute_error')
#DecisionTreeRegressor(max_features='auto', max_depth=3, citerion='friedman_mse')
#model.fit(X_train, y_train)
#y_pred = model.predict(X_val)
#mean_squared_error(np.log(y_val), np.log(y_pred))

In [23]:
# Apply optimal hyperparameters to the model
result = cross_validate(model, X_train, y_train, return_estimator=True)

In [24]:
max_index = result['test_score'].argmax()
model = result['estimator'][max_index]

In [25]:
output = model.predict(X_test)

In [26]:
index_array = np.zeros(np.shape(output))
for i in range(np.shape(index_array)[0]):
    index_array[i] = 1461+i
index_array = np.reshape(index_array, (np.shape(index_array)[0],1))
output = np.reshape(output, (np.shape(output)[0],1))

In [27]:
index = pd.DataFrame(index_array, columns = ['Id'], dtype=int)
sale = pd.DataFrame(output, columns = ['SalePrice'], dtype=float)
output_df = index.join(sale)

In [28]:
output_df.to_csv('output.csv',index=False)